In [7]:
import json
from collections.abc import MutableSequence
from pathlib import Path
from ipyshow.sudoku.util import dashboard,Sudoku

#--------------------------------------------------------------------------
class Dataset (MutableSequence[Sudoku]):
  r"""
Utity to manage a sample set of grids in a file. Current version limited to 9x9.
The file must contain a sequence of grid descriptors separated by an empty line.
Each grid descriptor is composed of one line of property descriptor followed by 9 lines, each composed of 9 characters from ".123456789".
Each character represents a cell ("." for the empty cell).
The property descriptor is a sequence of key:value pairs (with ":" in between) separated by ";". The value must be jsonable.

:param p: path to the file to parse
  """
#--------------------------------------------------------------------------
  tag2pos = {'.':-1}|{str(k+1):k for k in range(9)}
  pos2tag = {-1:'.'}|{k:str(k+1) for k in range(9)}
  def __init__(self,path): self.path = Path(path); self.load()
  def __len__(self): return len(self.content)
  def __getitem__(self,k): return self.content[k]
  def __setitem__(self,k,s):
    assert isinstance(s,Sudoku); s.solve(check=True)
    self.content[k] = s
    self.save()
  def __delitem__(self,k):
    del self.content[k]
    self.save()
  def insert(self,k,s):
    assert isinstance(s,Sudoku); s.solve(check=True)
    self.content.insert(k,s)
    self.save()
  def load(self,p=None):
    p = self.path if p is None else Path(p)
    self.content = [self.parse(a) for a in p.read_text().strip().split('\n\n')]
  def save(self,p=None):
    p = self.path if p is None else Path(p)
    p_sav = p.with_suffix('.sav'); p.replace(p_sav)
    p.write_text('\n\n'.join('\n'.join(self.unparse(s)) for s in self.content))
  def unparse(self,s):
    yield ';'.join(f'{k}: {json.dumps(v)}' for k,v in s.props.items())
    for r in s.grid: yield ''.join(self.pos2tag[k] for k in r)
  def parse(self,a):
    def kv(x): k,v = x.strip().split(':',1); return k.strip(),json.loads(v)
    label,*a = a.strip().split('\n')
    assert len(a)==9 and all(len(r)==9 for r in a)
    return Sudoku([[self.tag2pos[c] for c in r] for r in a],props=dict(kv(x) for x in label.strip().split(';')))

w = dashboard(Dataset('sudoku.txt'))
w